# 主キー、外部キーの設定

このノートブックでは、Yukiteru Martのデータベーステーブルに主キー（Primary Key）と外部キー（Foreign Key）の制約を設定します。

## 目的
- データの整合性を保つ
- 参照整合性を確保する
- テーブル間のリレーションシップを定義する

## 対象テーブル
1. **customers_table** - 顧客マスタ
2. **products_table** - 商品マスタ  
3. **stores_table** - 店舗マスタ
4. **transactions_table** - 取引履歴
5. **inventory_table** - 在庫管理




In [ ]:
# 環境設定
# スキーマ名を一括で管理するための変数
catalog = "users"
schema = "yukiteru_koide"
full_schema = f"{catalog}.{schema}"

# テーブル名を定義
customers_table = f"{full_schema}.customers_table"
products_table = f"{full_schema}.products_table"
stores_table = f"{full_schema}.stores_table"
transactions_table = f"{full_schema}.transactions_table"
inventory_table = f"{full_schema}.inventory_table"

print(f"スキーマ: {full_schema}")
print(f"顧客テーブル: {customers_table}")
print(f"商品テーブル: {products_table}")
print(f"店舗テーブル: {stores_table}")
print(f"取引テーブル: {transactions_table}")
print(f"在庫テーブル: {inventory_table}")


## 1. 顧客テーブル（customers_table）の制約設定

顧客テーブルに主キー制約を設定します。
- `customer_id` を NOT NULL に設定
- `customer_id` を主キーとして定義


In [0]:
# 顧客IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {customers_table}
ALTER COLUMN customer_id SET NOT NULL
""")

# 顧客IDを主キーとして設定
spark.sql(f"""
ALTER TABLE {customers_table}
ADD CONSTRAINT pk_customers PRIMARY KEY (customer_id)
""")


## 2. 商品テーブル（products_table）の制約設定

商品テーブルに主キー制約を設定します。
- `product_id` を NOT NULL に設定
- `product_id` を主キーとして定義


In [0]:
# 商品IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {products_table}
ALTER COLUMN product_id SET NOT NULL
""")

# 商品IDを主キーとして設定
spark.sql(f"""
ALTER TABLE {products_table}
ADD CONSTRAINT pk_products PRIMARY KEY (product_id)
""")


## 3. 店舗テーブル（stores_table）の制約設定

店舗テーブルに主キー制約を設定します。
- `store_id` を NOT NULL に設定
- `store_id` を主キーとして定義


In [0]:
# 店舗IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {stores_table}
ALTER COLUMN store_id SET NOT NULL
""")

# 店舗IDを主キーとして設定
spark.sql(f"""
ALTER TABLE {stores_table}
ADD CONSTRAINT pk_stores PRIMARY KEY (store_id)
""")


## 4. 取引テーブル（transactions_table）の制約設定

取引テーブルに主キーと外部キー制約を設定します。
- `transaction_id` を主キーとして設定
- `customer_id` を顧客テーブルへの外部キーとして設定
- `product_id` を商品テーブルへの外部キーとして設定
- `store_id` を店舗テーブルへの外部キーとして設定


In [0]:
# 取引IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {transactions_table}
ALTER COLUMN transaction_id SET NOT NULL
""")

# 取引IDを主キーとして設定
spark.sql(f"""
ALTER TABLE {transactions_table}
ADD CONSTRAINT pk_transactions PRIMARY KEY (transaction_id)
""")

# 顧客IDを外部キーとして設定（顧客テーブルを参照）
spark.sql(f"""
ALTER TABLE {transactions_table}
ADD CONSTRAINT fk_transactions_customer FOREIGN KEY (customer_id)
REFERENCES {customers_table}(customer_id)
""")

# 商品IDを外部キーとして設定（商品テーブルを参照）
spark.sql(f"""
ALTER TABLE {transactions_table}
ADD CONSTRAINT fk_transactions_product FOREIGN KEY (product_id)
REFERENCES {products_table}(product_id)
""")

# 店舗IDを外部キーとして設定（店舗テーブルを参照）
spark.sql(f"""
ALTER TABLE {transactions_table}
ADD CONSTRAINT fk_transactions_store FOREIGN KEY (store_id)
REFERENCES {stores_table}(store_id)
""")


## 5. 在庫テーブル（inventory_table）の制約設定

在庫テーブルに複合主キーと外部キー制約を設定します。
- `store_id` と `product_id` の組み合わせを複合主キーとして設定
- `store_id` を店舗テーブルへの外部キーとして設定
- `product_id` を商品テーブルへの外部キーとして設定


In [0]:
# 店舗IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {inventory_table}
ALTER COLUMN store_id SET NOT NULL
""")

# 商品IDをNOT NULL制約に設定
spark.sql(f"""
ALTER TABLE {inventory_table}
ALTER COLUMN product_id SET NOT NULL
""")

# 店舗IDと商品IDの複合主キーを設定
spark.sql(f"""
ALTER TABLE {inventory_table}
ADD CONSTRAINT pk_inventory PRIMARY KEY (store_id, product_id)
""")

# 店舗IDを外部キーとして設定（店舗テーブルを参照）
spark.sql(f"""
ALTER TABLE {inventory_table}
ADD CONSTRAINT fk_inventory_store FOREIGN KEY (store_id)
REFERENCES {stores_table}(store_id)
""")

# 商品IDを外部キーとして設定（商品テーブルを参照）
spark.sql(f"""
ALTER TABLE {inventory_table}
ADD CONSTRAINT fk_inventory_product FOREIGN KEY (product_id)
REFERENCES {products_table}(product_id)
""")


## まとめ

このノートブックで以下の制約が設定されました：

### 主キー制約
- `customers_table.customer_id`
- `products_table.product_id`
- `stores_table.store_id`
- `transactions_table.transaction_id`
- `inventory_table.(store_id, product_id)` - 複合主キー

### 外部キー制約
- `transactions_table.customer_id` → `customers_table.customer_id`
- `transactions_table.product_id` → `products_table.product_id`
- `transactions_table.store_id` → `stores_table.store_id`
- `inventory_table.store_id` → `stores_table.store_id`
- `inventory_table.product_id` → `products_table.product_id`

これらの制約により、データの整合性と参照整合性が保たれ、リレーショナルデータベースとしての品質が向上します。
